### logistic Regression

Se basa en la probabilidad condicionada que parte de la cantidad de casos favorables entre la cantidad de casos posibles sabiendo que a priori se cumplen ciertas condiciones.

- ¿Cómo se puede transformar una relacipon lineal a partir de una variable productora de probabilidades ?

Regresión lineal vs la regressión logística.

En la Regresión Lineal la variable de salida siempre es una variable continua, es decir, siempre es un número, por lo tanto la regresión lineal siempre es una buena opción cuando se quiere predecir un valor numérico contínuo. Sin embargo, ¿Qué ocurre si la variable no es continua?, es decir, cuando la variable es discreta, por ejemplo, que pasa si queremos clasificar nuestros registros en dos o más categorías, por ejemplo, un jugador de videojuegos online si es potencial pagador o no, si es potencial viralizador de publicar gratuitamente para obtener modenas a cambio o no, en esos casos, podemos extender los requisitos de un modelo de una relacióm lineal para intentar clasificar cada uno de los valores.

Con la regresión logística, básicamente se construye una variable de salida que es binaria, es decir, 0 o 1 o bien categórica en lugar de ser una variable contpinua.

### Implementación del método de la máxima verosimilitud para la Regresión Logistica

data bancaria, muestra cómo el cliente y el banco se relacionan entre si

In [1]:
# Tratamiento de los datos
###########################################################
import pandas as pd
import numpy as np
import datetime
import os

# Gráficos
###########################################################
import matplotlib.pyplot as plt
import seaborn as sns
#from ggplot import *

# Procesado y modelado
###########################################################
import statsmodels.api as sm
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate

# Configuración del warning
###########################################################
import warnings
warnings.filterwarnings('ignore')

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('../Data/bank.csv',sep=';',on_bad_lines='skip')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,periodo
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,999,0,nonexistent,"-1,8",92.893,"-46,2",1.313,"5099,1",no,202012
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,999,0,nonexistent,"1,1",93.994,"-36,4",4.855,5191,no,202012
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,999,0,nonexistent,"1,4",94.465,"-41,8",4.962,"5228,1",no,202012
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,999,0,nonexistent,"1,4",94.465,"-41,8",4.959,"5228,1",no,202012
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,999,0,nonexistent,"-0,1","93,2",-42,4.191,"5195,8",no,202012


In [3]:
df.shape

(4119, 22)

In [4]:
#Listo el nombre de las columnas
df.columns.values

array(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y', 'periodo'],
      dtype=object)

In [5]:
#Convierto los datos de la columna Y en binario
df['y'] = (df['y'] == 'yes').astype(int)

In [6]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,periodo
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,999,0,nonexistent,"-1,8",92.893,"-46,2",1.313,"5099,1",0,202012
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,999,0,nonexistent,"1,1",93.994,"-36,4",4.855,5191,0,202012
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,999,0,nonexistent,"1,4",94.465,"-41,8",4.962,"5228,1",0,202012
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,999,0,nonexistent,"1,4",94.465,"-41,8",4.959,"5228,1",0,202012
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,999,0,nonexistent,"-0,1","93,2",-42,4.191,"5195,8",0,202012


In [7]:
# Evalúo la columna grado educativo ya que puede ser muy importante para el modleo final
df["education"].unique()

array(['basic.9y', 'high.school', 'university.degree',
       'professional.course', 'basic.6y', 'basic.4y', 'unknown',
       'illiterate'], dtype=object)

In [8]:
#Agrupo las categprías en una nueva columna
df["education"] = np.where(df["education"]=="basic.4y", "Basic", df["education"])
df["education"] = np.where(df["education"]=="basic.6y", "Basic", df["education"])
df["education"] = np.where(df["education"]=="basic.9y", "Basic", df["education"])

df["education"] = np.where(df["education"]=="high.school", "High School", df["education"])
df["education"] = np.where(df["education"]=="professional.course", "Professional Course", df["education"])
df["education"] = np.where(df["education"]=="university.degree", "University Degree", df["education"])

df["education"] = np.where(df["education"]=="illiterate", "Illiterate", df["education"])
df["education"] = np.where(df["education"]=="unknown", "Unknown", df["education"])

In [9]:
#Verifico que la nueva columna 
df["education"].unique()

array(['Basic', 'High School', 'University Degree', 'Professional Course',
       'Unknown', 'Illiterate'], dtype=object)

In [10]:
#Análisis exploratorio de los datos
df['y'].value_counts()

0    3668
1     451
Name: y, dtype: int64

- No compraron el producto 3.668 personas.
- Si compraron el producto 451 personas.

In [11]:
#Análisis de variables numéricas
df.groupby("y").mean().round()

,age,duration,campaign,pdays,previous,periodo
y,,,,,,
0,40.0,219.0,3.0,983.0,0.0,202130.0
1,42.0,561.0,2.0,779.0,1.0,202129.0


### Conclusiones previas análisis variables numéricas.

- El promedio de edad de los clientes que compran el producto (42 años) es mayor a los que no lo compran (40 años)
- El promedio de segundos desde el último contacto (duration) es mayor para los clientes que compran el producto.
- El promedio del npumero de contactos efectivos es menor en los clientes que compran el producto.
- La duración en días desde que el cliente fue contactado (pdays) de los clientes que compran el producto es menor a la duración de los clientes que no compran el producto.
- El promedio de contactos previos efectivos para los clientes que no compran el producto es cero.

- En general, se aprecia que la edad es una variable que pesa mucho a la hora de adquirir un producto bancario,mientras que en promedio, los clientes cuyos contactos efectivos previos a la contratación del producto son efectivos, es determinante para la adquisicón del mismo.

In [12]:
#Analizo promedios para las variables numéricas por nivel de educación
df.groupby('education').mean().round().reset_index()

,education,age,duration,campaign,pdays,previous,y,periodo
0,Basic,42.0,254.0,2.0,979.0,0.0,0.0,202130.0
1,High School,38.0,259.0,3.0,958.0,0.0,0.0,202129.0
2,Illiterate,42.0,146.0,4.0,999.0,0.0,0.0,202205.0
3,Professional Course,40.0,279.0,3.0,958.0,0.0,0.0,202133.0
4,University Degree,39.0,248.0,3.0,948.0,0.0,0.0,202130.0
5,Unknown,43.0,267.0,3.0,940.0,0.0,0.0,202117.0


In [ ]:
#Analizo con un gráfico la relación entre el nivel de educación y la columna y
%matplotlib inline
pd.crosstab(df.education, df.y).plot(kind="bar")
plt.title("Frecuencia de compra en función del nivel de educación")
plt.xlabel("Nivel de educación")
plt.ylabel("Frecuencia de compra del producto")

En el gráfico se aprecia como los clientes que más compran productos son aquellos con grado universitario (barra naranja), por lo que la variable nivel de educación parece ser un buen predictor.

In [ ]:
table=pd.crosstab(df.marital, df.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True)
plt.title("Diagrama apilado de estado civil contra el nivel de compras")
plt.xlabel("Estado civil")
plt.ylabel("Proporción de clientes")

Las personas solteras (single) son las que más compran productos bancarios, sin embargo la diferencia no es representativa con respecto a otros estados civiles, por lo que probablemente esta variable sea poco predictiva.

In [ ]:
%matplotlib inline
table= pd.crosstab(df.day_of_week, df.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True)
plt.title("Frecuencia de compra en función del día de la semana")
plt.xlabel("Día de la semana")
plt.ylabel("Frecuencia de compra del producto")

In [ ]:
%matplotlib inline
table= pd.crosstab(df.month, df.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True)
plt.title("Frecuencia de compra en función del mes")
plt.xlabel("Mes del año")
plt.ylabel("Frecuencia de compra del producto")

Parece ser que los meses de diciembre y marzo representan mayores ventas, sin embargo a priori no sabemos si eso se debe a que en esos meses hay menos registros.

In [ ]:
%matplotlib inline
table.plot(kind="bar", stacked=False)
plt.title("Frecuencia de compra en función del mes")
plt.xlabel("Mes del año")
plt.ylabel("Frecuencia de compra del producto")

Muchos clientes rechazan los productos durante el mes de mayo.

In [ ]:
%matplotlib inline
df.age.hist()
plt.title("Histograma de la Edad")
plt.xlabel("Edad")
plt.ylabel("Cliente")

La distribución de la edad muestra que la mayoría de los clientes tienen entre 30 y 40 años

In [ ]:
pd.crosstab(df.age, df.y).plot(kind="bar")

Observando la proporción de productos comprados por edad, se evidencia como los clientes entre 30 y 40 años son los que más compran.

In [ ]:
pd.crosstab(df.poutcome, df.y).plot(kind="bar")

- los clientes que rechazaron el producto en el pasado, suelen rechazar el producto en el presente a una tasa mayor que en el futuro.

- Los clientes que compraron productos en el pasado, suelen comprar en el presente y en el futuro, auqnue este último a una tasa menor.

### Conversión de las variables categóricas a dummies

In [13]:
categories = ["job", "marital", "education", "housing", "loan", "contact", 
              "month", "day_of_week", "poutcome","euribor3m"]
for category in categories:
    cat_list = "cat"+ "_"+category
    cat_dummies = pd.get_dummies(df[category], prefix=category)
    df_new = df.join(cat_dummies)
    df = df_new

In [14]:
data_vars = df.columns.values.tolist()

In [15]:
to_keep = [v for v in data_vars if v not in categories]
to_keep = [v for v in to_keep if v not in ["default"]]

In [16]:
bank_data = df[to_keep]
bank_data.head()

,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,nr_employed,y,...,euribor3m_4.959,euribor3m_4.961,euribor3m_4.962,euribor3m_4.963,euribor3m_4.964,euribor3m_4.965,euribor3m_4.966,euribor3m_4.967,euribor3m_4.968,euribor3m_5.045
0,30,487,2,999,0,"-1,8",92.893,"-46,2","5099,1",0,...,0,0,0,0,0,0,0,0,0,0
1,39,346,4,999,0,"1,1",93.994,"-36,4",5191,0,...,0,0,0,0,0,0,0,0,0,0
2,25,227,1,999,0,"1,4",94.465,"-41,8","5228,1",0,...,0,0,1,0,0,0,0,0,0,0
3,38,17,3,999,0,"1,4",94.465,"-41,8","5228,1",0,...,1,0,0,0,0,0,0,0,0,0
4,47,58,1,999,0,"-0,1","93,2",-42,"5195,8",0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
bank_data.columns.values

array(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'nr_employed', 'y', 'periodo',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired',
       'job_self-employed', 'job_services', 'job_student',
       'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_Basic', 'education_High School',
       'education_Illiterate', 'education_Professional Course',
       'education_University Degree', 'education_Unknown', 'housing_no',
       'housing_unknown', 'housing_yes', 'loan_no', 'loan_unknown',
       'loan_yes', 'contact_cellular', 'contact_telephone', 'month_apr',
       'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar',
       'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day

### Implementación del modelo con la librería Statsmodel.api

In [22]:
#Selecciono variables de acuerdo al análisis estadistico
cols = ["previous", "job_blue-collar", "job_retired", "month_aug", "month_dec", 
        "month_jul", "month_jun", "month_mar", "month_nov", "day_of_week_wed", "poutcome_nonexistent"]

In [23]:
X = bank_data[cols]
Y = bank_data['y']

In [24]:
logit_model = sm.Logit(Y, X)

In [25]:
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.324471
         Iterations 7


In [26]:
result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                           Results: Logit
=====================================================================
Model:                Logit             Pseudo R-squared:  0.061     
Dependent Variable:   y                 AIC:               2694.9922 
Date:                 2023-02-10 12:15  BIC:               2764.5492 
No. Observations:     4119              Log-Likelihood:    -1336.5   
Df Model:             10                LL-Null:           -1422.9   
Df Residuals:         4108              LLR p-value:       7.1214e-32
Converged:            1.0000            Scale:             1.0000    
No. Iterations:       7.0000                                         
---------------------------------------------------------------------
                      Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
---------------------------------------------------------------------
previous             -0.3631   0.0727  -4.9935 0.0000 -0.5056 -0.2206
job_blue-collar      -0.7678   0.1488  -5.1597 0.0000 -1.0595 -0.4762
job_retired           0.4797   0.2126   2.2568 0.0240  0.0631  0.8963
month_aug            -0.1322   0.1587  -0.8330 0.4048 -0.4433  0.1788
month_dec             1.5490   0.4865   3.1839 0.0015  0.5955  2.5026
month_jul            -0.1142   0.1621  -0.7047 0.4810 -0.4320  0.2035
month_jun             0.3692   0.1580   2.3371 0.0194  0.0596  0.6788
month_mar             2.0259   0.3299   6.1407 0.0000  1.3793  2.6725
month_nov            -0.6228   0.1810  -3.4415 0.0006 -0.9775 -0.2681
day_of_week_wed      -0.1438   0.1339  -1.0735 0.2831 -0.4063  0.1187
poutcome_nonexistent -2.2827   0.0958 -23.8185 0.0000 -2.4705 -2.0949
=====================================================================

"""

### Implementación del modelo con la librería scikit-learn

In [28]:
from sklearn import linear_model
logit_model = linear_model.LogisticRegression()
logit_model.fit(X,Y)

LogisticRegression()

In [29]:
logit_model.score(X,Y)

0.8948773974265598

In [30]:
1-Y.mean()

0.8905074047098811

In [31]:
pd.DataFrame(list(zip(X.columns, np.transpose(logit_model.coef_))))

,0,1
0,previous,[0.7204356322386244]
1,job_blue-collar,[-0.49769383225588193]
2,job_retired,[0.6319429244561681]
3,month_aug,[0.01585013762156673]
4,month_dec,[1.5950349880667416]
5,month_jul,[0.00610341596471674]
6,month_jun,[0.46875792942059585]
7,month_mar,[2.00514592151791]
8,month_nov,[-0.199569908288184]
9,day_of_week_wed,[0.025689317399753133]


## Validación del modelo logístico

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state=0)

In [34]:
lm = linear_model.LogisticRegression()
lm.fit(X_train, Y_train)

LogisticRegression()

In [36]:
from IPython.display import display, Math, Latex
display(Math(r'Y_p=\begin{cases}0& si\ p\leq0.5\\1&si\ p >0.5\end{cases}'))

<IPython.core.display.Math object>

In [37]:
probs = lm.predict_proba(X_test)

In [38]:
probs

array([[0.92461341, 0.07538659],
       [0.91562776, 0.08437224],
       [0.87940301, 0.12059699],
       ...,
       [0.87912618, 0.12087382],
       [0.91562776, 0.08437224],
       [0.38204631, 0.61795369]])

In [39]:
prediction = lm.predict(X_test)

In [40]:
prediction

array([0, 0, 0, ..., 0, 0, 1])

In [41]:
display(Math(r'\varepsilon\in (0,1), Y_p=\begin{cases}0& si\ p\leq \varepsilon\\1&si\ p >\varepsilon\end{cases}'))

<IPython.core.display.Math object>

In [42]:
prob = probs[:,1]
prob_df = pd.DataFrame(prob)
threshold = 0.1
prob_df["prediction"] = np.where(prob_df[0]>threshold, 1, 0)
prob_df.head()

,0,prediction
0,0.075387,0
1,0.084372,0
2,0.120597,1
3,0.083494,0
4,0.040698,0


In [43]:
pd.crosstab(prob_df.prediction, columns="count")

col_0,count
prediction,
0,931
1,305


In [44]:
390/len(prob_df)*100

31.55339805825243

In [45]:
threshold = 0.15
prob_df["prediction"] = np.where(prob_df[0]>threshold, 1, 0)
pd.crosstab(prob_df.prediction, columns="count")

col_0,count
prediction,
0,1092
1,144


In [46]:
331/len(prob_df)*100

26.779935275080906

In [47]:
threshold = 0.05
prob_df["prediction"] = np.where(prob_df[0]>threshold, 1, 0)
pd.crosstab(prob_df.prediction, columns="count")

col_0,count
prediction,
0,28
1,1208


In [48]:
732/len(prob_df)*100

59.22330097087378

In [49]:
metrics.accuracy_score(Y_test, prediction)

0.8996763754045307

## Validación cruzada

In [51]:
scores = cross_validate(linear_model.LogisticRegression(), X, Y, scoring="accuracy", cv=10)

In [52]:
scores

{'fit_time': array([0.02771187, 0.02196288, 0.02239275, 0.02219009, 0.07382703,
        0.01810789, 0.01513386, 0.01618838, 0.01591611, 0.01459098]),
 'score_time': array([0.00277781, 0.00266027, 0.00276303, 0.00284505, 0.00237799,
        0.002002  , 0.00243711, 0.00258088, 0.00202918, 0.00192308]),
 'test_score': array([0.89320388, 0.88834951, 0.90291262, 0.89805825, 0.89563107,
        0.8907767 , 0.8907767 , 0.89563107, 0.8907767 , 0.89294404])}

In [53]:
scores.mean()

AttributeError: 'dict' object has no attribute 'mean'